In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping

# load class_dict.csv & Bird Species.csv

In [ ]:
bird_classes = pd.read_csv('../input/100-bird-species/class_dict.csv')
bird_species = pd.read_csv('../input/100-bird-species/Bird Species.csv')
bird_species.sample(5)
#bird_classes

# Show All Bird Species

In [ ]:
birdTypes = os.listdir('../input/100-bird-species/train')
len(birdTypes)

create bird species class

In [ ]:
species = np.array(bird_classes['class'])
#species

In [ ]:
def show_sample(df, num, species):
    df = df.sample(12)
    plt.figure(figsize=(20,20))
    for i in range(len(df)):
        #print(df.iloc[i]['filepaths'])
        path = '../input/100-bird-species/' + df.iloc[i]['filepaths']
        label = df.iloc[i]['labels']
        Type = df.iloc[i]['data set']
        plt.subplot(3, 4, i+1)
        img = tf.keras.preprocessing.image.load_img(path)
        plt.imshow(img)
        plt.xlabel(f'{label} : {Type}')
    plt.show()
        

# Visualize bird image from sample

In [ ]:
show_sample(bird_species, 5, bird_species)

# Load train data & valid data
we can see train data includes 43622 images belonging to 300 species and valid data includes 1500 images belonging to 300 species
I set preprocessing with preprocess_input of VGG16 and doing Data Augmentation without rescale 1/255. (i tried to rescale but VGG16 model is training very slow. if anyone know why it's very slowly, please comment)

In [ ]:
train_data = ImageDataGenerator(preprocessing_function = preprocess_input, rotation_range=20, horizontal_flip=True)
train_generator = train_data.flow_from_directory('../input/100-bird-species/train',
                                                batch_size=64,
                                                target_size=(224,224),
                                                class_mode='categorical')

In [ ]:
valid_data = ImageDataGenerator(preprocessing_function = preprocess_input, rotation_range=20, horizontal_flip=True)
validation_generator = valid_data.flow_from_directory('../input/100-bird-species/valid',
                                                batch_size=64,
                                                target_size=(224,224),
                                                class_mode='categorical')

# Visulaize image after preprocessing by VGG16

In [ ]:
for _ in range(5):
    img, label = train_generator.next()
    print(img.shape) 
    plt.imshow(img[0])
    plt.show()

# Create a model
First, I create VGG16 model and set vgg_model.trainable = False for warming up model

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
#eff_model.summary()
vgg_model.trainable = False

layer0 = tf.keras.layers.Flatten(name='flatten')(vgg_model.output)
layer1 = tf.keras.layers.Dense(4096, activation='relu',name='fc1')(layer0)
layer2 = tf.keras.layers.Dense(4096, activation='relu',name='fc2')(layer1)
out_layer = tf.keras.layers.Dense(300, activation='softmax')(layer2)
vgg_model = tf.keras.Model(vgg_model.input, out_layer)
vgg_model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
vgg_model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
              #,
              #ModelCheckpoint(filepath='Fullmodel.{epoch:02d}-{acc:.2f}.h5', monitor='val_acc', save_best_only=False, mode='max')]
             ]

In [ ]:
history = vgg_model.fit(
      train_generator, 
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      callbacks=callbacks)

Next, I set vgg_model.trainable = True to train all model after warming up and decrease learning rate (0.0001 -> 0.00005)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
vgg_model.trainable = True
vgg_model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
              #,
              #ModelCheckpoint(filepath='Fullmodel.{epoch:02d}-{acc:.2f}.h5', monitor='val_acc', save_best_only=False, mode='max')]
             ]

In [ ]:
history = vgg_model.fit(
      train_generator, 
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      callbacks=callbacks)

Last, I tried to decrease learning rate again (0.00005 -> 0.00001)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
vgg_model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)
              #,
              #ModelCheckpoint(filepath='Fullmodel.{epoch:02d}-{acc:.2f}.h5', monitor='val_acc', save_best_only=False, mode='max')]
             ]

In [ ]:
history = vgg_model.fit(
      train_generator, 
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
test = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test.flow_from_directory(
        '../input/100-bird-species/test',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical')

Check ACC by test_generator

In [ ]:
vgg_model.evaluate(test_generator,use_multiprocessing=True,workers=10)

***Please forgive any errors/mistakes.***
***you can suggest me in comment thanks!***